In [1]:
import pandas as pd
import numpy as np

import yfinance as yf
from fredapi import Fred

import requests
import json

In [2]:
api_key='7ae621c107271d79c66cddcfd2ba6a3d'
start = '2000-01-01'
end = '2023-06-01'

In [3]:
fred = Fred(api_key=api_key)

unemployment_rate = pd.DataFrame(fred.get_series(
    'UNRATE',
    observation_start=start,
    observation_end=end)) #resample("M").mean()

unemployment_rate = unemployment_rate.reset_index(drop=False)
unemployment_rate.columns = ['Date', 'unemployment_rate']
unemployment_rate

,Date,unemployment_rate
0,2000-01-01,4.0
1,2000-02-01,4.1
2,2000-03-01,4.0
3,2000-04-01,3.8
4,2000-05-01,4.0
...,...,...
275,2022-12-01,3.5
276,2023-01-01,3.4
277,2023-02-01,3.6
278,2023-03-01,3.5


In [4]:
inflation_rate = pd.DataFrame(fred.get_series(
    'CPIAUCSL',
    observation_start=start,
    observation_end=end))

inflation_rate = inflation_rate.reset_index(drop=False)
inflation_rate.columns = ['Date', 'inflation_rate']
inflation_rate

,Date,inflation_rate
0,2000-01-01,169.300
1,2000-02-01,170.000
2,2000-03-01,171.000
3,2000-04-01,170.900
4,2000-05-01,171.200
...,...,...
275,2022-12-01,298.990
276,2023-01-01,300.536
277,2023-02-01,301.648
278,2023-03-01,301.808


In [5]:
treasury_yield = pd.DataFrame(fred.get_series(
    'DGS10',
    observation_start=start,
    observation_end=end))

treasury_yield = treasury_yield.reset_index(drop=False)
treasury_yield.columns = ['Date', 'treasury_yield']
ty_grouped = treasury_yield.set_index('Date').groupby(pd.Grouper(freq='M')).mean()
ty_grouped = ty_grouped.reset_index()
ty_grouped['Date'] = ty_grouped['Date'] + pd.offsets.MonthBegin(1)
ty_grouped

,Date,treasury_yield
0,2000-02-01,6.661000
1,2000-03-01,6.519500
2,2000-04-01,6.256522
3,2000-05-01,5.990526
4,2000-06-01,6.440455
...,...,...
276,2023-02-01,3.531500
277,2023-03-01,3.746842
278,2023-04-01,3.663043
279,2023-05-01,3.460000


In [6]:
# https://www.spglobal.com/marketintelligence/en/mi/products/us-monthly-gdp-index.html
gdp = pd.read_csv("S&P_MGDP.csv")
gdp

,Unnamed: 0,Monthly Nominal GDP Index,Monthly Real GDP Index
0,1992 - Jan,6315.522893,9485.743414
1,1992 - Feb,6356.726995,9528.069786
2,1992 - Mar,6417.056111,9606.300557
3,1992 - Apr,6443.171182,9612.356062
4,1992 - May,6433.783856,9594.443772
...,...,...,...
369,2022 - Oct,26028.626520,20127.400700
370,2022 - Nov,26231.133840,20222.026550
371,2022 - Dec,26154.100680,20204.805190
372,2023 - Jan,26427.420140,20283.655220


In [7]:
def date_maker(df):
    df['Date'] = pd.to_datetime(df['Date'])

    # Extract the year and month from the datetime
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = 1

    # Set the day as 1 for all rows
    df['Date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])
    return df

In [8]:
gdp.columns = ['Date', 'Monthly Nominal GDP Index', 'Monthly Real GDP Index']
gdp = date_maker(gdp)
gdp

,Date,Monthly Nominal GDP Index,Monthly Real GDP Index,Year,Month,Day
0,1992-01-01,6315.522893,9485.743414,1992,1,1
1,1992-02-01,6356.726995,9528.069786,1992,2,1
2,1992-03-01,6417.056111,9606.300557,1992,3,1
3,1992-04-01,6443.171182,9612.356062,1992,4,1
4,1992-05-01,6433.783856,9594.443772,1992,5,1
...,...,...,...,...,...,...
369,2022-10-01,26028.626520,20127.400700,2022,10,1
370,2022-11-01,26231.133840,20222.026550,2022,11,1
371,2022-12-01,26154.100680,20204.805190,2022,12,1
372,2023-01-01,26427.420140,20283.655220,2023,1,1


In [9]:
gdp = gdp[['Date', 'Monthly Real GDP Index']]

In [10]:
gdp['GDP_growth'] = (gdp['Monthly Real GDP Index'] / gdp['Monthly Real GDP Index'].shift(1) - 1) * 100
gdp = gdp.iloc[1:]
gdp = gdp[gdp['Date'] >= start]
gdp

C:\Users\HP\AppData\Local\Temp\ipykernel_15892\2763604364.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gdp['GDP_growth'] = (gdp['Monthly Real GDP Index'] / gdp['Monthly Real GDP Index'].shift(1) - 1) * 100


,Date,Monthly Real GDP Index,GDP_growth
96,2000-01-01,12870.47162,-1.066967
97,2000-02-01,12934.38121,0.496560
98,2000-03-01,13000.50928,0.511258
99,2000-04-01,13173.13041,1.327803
100,2000-05-01,13149.18201,-0.181797
...,...,...,...
369,2022-10-01,20127.40070,0.280228
370,2022-11-01,20222.02655,0.470134
371,2022-12-01,20204.80519,-0.085161
372,2023-01-01,20283.65522,0.390254


In [11]:
sp500 = yf.download('SPY', start=start, end=end, interval='1mo')
sp500

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2000-01-01,148.250000,148.250000,135.000000,139.562500,91.111023,156770800
2000-02-01,139.750000,144.562500,132.718750,137.437500,89.723793,186938300
2000-03-01,137.625000,155.750000,135.031250,150.375000,98.169800,247594900
2000-04-01,150.125000,153.109375,133.500000,145.093750,94.962761,229246200
2000-05-01,146.562500,148.484375,136.500000,142.812500,93.469749,161024000
...,...,...,...,...,...,...
2023-01-01,384.369995,408.160004,377.829987,406.480011,404.934570,1575450100
2023-02-01,405.209991,418.309998,393.640015,396.260010,394.753418,1603094700
2023-03-01,395.410004,409.700012,380.649994,409.390015,407.833527,2515995800


In [12]:
sp500['SP500_return'] = (sp500['Adj Close'] / sp500['Adj Close'].shift(1) - 1) * 100 
sp500 = sp500.iloc[1:]
sp500 = sp500.reset_index()
sp500 = sp500[['Date', 'SP500_return']]
sp500

,Date,SP500_return
0,2000-02-01,-1.522571
1,2000-03-01,9.413341
2,2000-04-01,-3.266828
3,2000-05-01,-1.572207
4,2000-06-01,1.728613
...,...,...
275,2023-01-01,6.776820
276,2023-02-01,-2.514271
277,2023-03-01,3.313488
278,2023-04-01,1.985238


In [13]:
macros = pd.merge(unemployment_rate, inflation_rate, on='Date', how='outer').merge(ty_grouped, on='Date', how='outer').merge(gdp, on='Date', how='outer').merge(sp500, on='Date', how='outer')
macros

,Date,unemployment_rate,inflation_rate,treasury_yield,Monthly Real GDP Index,GDP_growth,SP500_return
0,2000-01-01,4.0,169.300,NaN,12870.47162,-1.066967,NaN
1,2000-02-01,4.1,170.000,6.661000,12934.38121,0.496560,-1.522571
2,2000-03-01,4.0,171.000,6.519500,13000.50928,0.511258,9.413341
3,2000-04-01,3.8,170.900,6.256522,13173.13041,1.327803,-3.266828
4,2000-05-01,4.0,171.200,5.990526,13149.18201,-0.181797,-1.572207
...,...,...,...,...,...,...,...
277,2023-02-01,3.6,301.648,3.531500,20193.96427,-0.442183,-2.514271
278,2023-03-01,3.5,301.808,3.746842,NaN,NaN,3.313488
279,2023-04-01,3.4,302.918,3.663043,NaN,NaN,1.985238
280,2023-05-01,NaN,NaN,3.460000,NaN,NaN,0.461619


In [14]:
macros.to_csv('macros.csv')

In [15]:
def get_data(tickers, start_date, end_date):

    """
    This function takes a list of stock tickers and return a data frame with features:
    - start date: yyyy-mm-dd
    - end date: yyyy-mm-dd
    - tickers: list of tickers

    Features:
    - Adj Closing Price and lagged Adj Closing Price to get Return (in percent)
    - High and Low because traders might set Sell or Buy commands
    - Rolling VaR(5) of a rolling week (5 trading days)
    - target: summed Rolling VaR(5) of a rolling week (5 trading days)

    We may change the features later
    """
    # initialise list to store ticker data farmes
    dfs = []

    for ticker in tickers:
        # Download the stock price data with yfinance
        data = yf.download(ticker, start=start_date, end=end_date, interval='1mo')
    
        # Create a new data frame with the necessary columns
        df = pd.DataFrame(index=data.index)
        df["ticker"] = ticker
        df["adj_close"] = data["Adj Close"]
        df["adj_close_lag"] = data["Adj Close"].shift(1)
        df["return"] = ((df["adj_close"] / df["adj_close_lag"]) - 1)*100
        df = df[["ticker", "return"]]
        # df["volume"] = data["Volume"]
        # df["high"] = data["High"]
        # df["low"] = data["Low"]
        # df["rolling_var_5"] = df["return"].rolling(5, min_periods=1).quantile(0.05)
        # # target is the cummulative 
        # df["target"] = df["rolling_var_5"].rolling(5, min_periods=5).sum()
    
        # Append the data frame to the list
        dfs.append(df)

    # Concatenate the data frames vertically
    result = pd.concat(dfs)
    result = result.dropna()

    return result

In [16]:
ticker = ['NVS', 'ABBV', 'AZN', 'BMY', 'JNJ', 'LLY', 'MRK', 'NVO', 'PFE', 'ROG']

In [17]:
stocks = get_data(ticker, start, end)
stocks

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,ticker,return
Date,,
2000-02-01,NVS,3.838400
2000-03-01,NVS,6.420222
2000-04-01,NVS,2.559430
2000-05-01,NVS,5.169303
2000-06-01,NVS,8.474611
...,...,...
2023-01-01,ROG,16.968326
2023-02-01,ROG,5.451681
2023-03-01,ROG,11.025813


In [19]:
stocks.reset_index(inplace=True)

# Pivot the data frame to invert it
inverted_df = stocks.pivot(index='Date', columns='ticker', values='return')

# Reset the column index
inverted_df.columns.name = None

inverted_df = inverted_df.reset_index()
inverted_df

,Date,ABBV,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG
0,2000-02-01,NaN,-12.828964,-13.227974,-16.339870,-11.121516,-21.701146,2.220002,3.838400,-11.226193,54.440789
1,2000-03-01,NaN,22.264136,-0.218321,-2.079029,5.804315,0.913734,8.390928,6.420222,14.102001,6.922258
2,2000-04-01,NaN,5.567379,-8.205707,17.437721,23.153662,12.400724,-0.097656,2.559430,15.213679,7.370518
3,2000-05-01,NaN,-0.148388,5.395756,8.484816,-1.296587,7.374090,20.863985,5.169303,5.637955,-8.163265
4,2000-06-01,NaN,10.549792,5.788878,14.239921,31.641733,3.078632,2.813679,8.474611,8.076056,13.131313
...,...,...,...,...,...,...,...,...,...,...,...
275,2023-01-01,-8.576205,-3.584079,0.972908,-7.489376,-5.928823,-2.549213,2.541749,-0.110227,-13.817335,16.968326
276,2023-02-01,5.152609,-0.290649,-4.328217,-6.217115,-9.568503,-1.089288,1.592445,-7.172811,-7.286115,5.451681
277,2023-03-01,3.554254,8.035329,0.507544,1.862736,10.703401,0.141189,12.873250,9.367574,0.566924,11.025813
278,2023-04-01,-5.176628,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467


In [20]:
final_df = pd.merge(macros, inverted_df, on='Date', how='outer')
final_df

,Date,unemployment_rate,inflation_rate,treasury_yield,Monthly Real GDP Index,GDP_growth,SP500_return,ABBV,AZN,BMY,JNJ,LLY,MRK,NVO,NVS,PFE,ROG
0,2000-01-01,4.0,169.300,NaN,12870.47162,-1.066967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-02-01,4.1,170.000,6.661000,12934.38121,0.496560,-1.522571,NaN,-12.828964,-13.227974,-16.339870,-11.121516,-21.701146,2.220002,3.838400,-11.226193,54.440789
2,2000-03-01,4.0,171.000,6.519500,13000.50928,0.511258,9.413341,NaN,22.264136,-0.218321,-2.079029,5.804315,0.913734,8.390928,6.420222,14.102001,6.922258
3,2000-04-01,3.8,170.900,6.256522,13173.13041,1.327803,-3.266828,NaN,5.567379,-8.205707,17.437721,23.153662,12.400724,-0.097656,2.559430,15.213679,7.370518
4,2000-05-01,4.0,171.200,5.990526,13149.18201,-0.181797,-1.572207,NaN,-0.148388,5.395756,8.484816,-1.296587,7.374090,20.863985,5.169303,5.637955,-8.163265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277,2023-02-01,3.6,301.648,3.531500,20193.96427,-0.442183,-2.514271,5.152609,-0.290649,-4.328217,-6.217115,-9.568503,-1.089288,1.592445,-7.172811,-7.286115,5.451681
278,2023-03-01,3.5,301.808,3.746842,NaN,NaN,3.313488,3.554254,8.035329,0.507544,1.862736,10.703401,0.141189,12.873250,9.367574,0.566924,11.025813
279,2023-04-01,3.4,302.918,3.663043,NaN,NaN,1.985238,-5.176628,5.489119,-3.664707,5.612908,15.269915,9.289214,5.836894,16.334413,-4.681371,-1.517467
280,2023-05-01,NaN,NaN,3.460000,NaN,NaN,0.461619,-7.868480,-0.191204,-2.695194,-5.277949,8.487855,-4.382088,-3.967915,-6.161645,-2.237080,-2.162160


In [21]:
final_df.to_csv('monthly_data.csv')